In [ ]:
import os
import pandas as pd
import torch as tc
import numpy as np
import matplotlib as plt
import datetime as dttt
import dateutil.parser as dtpr

pd.set_option('display.max_columns',500)

In [ ]:
target_csv = "data\\2023.csv"
if os.path.isfile(target_csv):
    data = pd.read_csv(target_csv)
else:
    raise FileNotFoundError()

In [ ]:
split_loc = "data\\set"
if not os.path.exists(split_loc):
    os.makedirs(split_loc)

In [ ]:
data.dtypes

### Clean up data

In [ ]:
data.drop(columns=["region","comment","num"], inplace=True)
data.sort_values(by="date",inplace=True)
data = data[data.secs != "-"]
data = data[data["or"] != "–"]
data = data[data.rpr != "–"]
data = data[data["pos"].str.isdigit()]
data["pos"] = data["pos"].astype(int)
data["class"] = np.where(~data["pattern"].isna(), "g", data["class"])
name_match = ["Classic Mile", "Classic Cup", "Derby"]
mapCL = {"Class 5": 0, "Class 4":0, "Class 3":1, "Class 2": 2, "Class 1":3, "l":4, "g":5, "NaN":0}
data["class"] = np.where(data["class"].isna() & data["race_name"].str.contains('|'.join(name_match)),"l",data["class"])
data["class"].fillna(value="NaN")
data["class_code"] = data["class"].replace(mapCL)
mapC = {"Happy Valley (HK)":0, "Sha Tin (HK)":1}
data["course_code"] = data["course"].replace(mapC)
data["date_time"] = pd.to_datetime(data["date"] + " " + data["off"])
data["date"] = pd.to_datetime(data["date"])
data["quarter"] = (data["date"].dt.month+2)//3
mapPT = {"N":0,"Group 3":1,"Group 2": 2, "Group 1":3}
data["pattern_code"] = (data["pattern"].fillna(value="N")).replace(mapPT)
mapAB = {"2yo+":0,"3yo":1,"3yo+":2,"4yo":3,"4yo+":4}
data["age_band_code"] = data["age_band"].replace(mapAB)
mapDist = {distance: i for i, distance in enumerate(sorted(data["dist_m"].unique()))}
data["dist_code"] = data["dist_m"].replace(mapDist)
mapGO = {surf: i for i, surf in enumerate(data["going"].unique())}
data["going_code"] = data["going"].replace(mapGO)
mapSF = {surf: i for i, surf in enumerate(data["surface"].unique())}
data["surface_code"] = data["surface"].replace(mapSF)
mapSX = {sex: i for i, sex in enumerate(data["sex"].unique())}
data["sex_code"] = data["sex"].replace(mapSX)
data.fillna(value={"hg":"None","class":"g"},inplace=True)

### Create new features

In [ ]:
data["in_place"] = np.where(data["pos"] <= 3,1,0)
data["win"] = np.where(data["pos"] == 1, 1, 0)
data["decLog"] = np.log1p(data["dec"])
data["age_u3"] = np.where(data["age"] <= 4, 1, 0)
data["rest_time"] = (data["date"]-(data.groupby("horse")["date"].shift())).dt.days
data["rest_less14"] = np.where(data["rest_time"] < 14, 1, 0)
data["rest_ovr32"] = (data["rest_time"]>32)*1
data["h_cwin"] = (data.groupby("horse")["win"].transform("sum"))/(data.groupby("horse")["horse"].transform("count"))
data["h_rwin"] = (data.groupby("horse", group_keys=False)["win"].apply(lambda x: x.rolling(4, min_periods=1).sum())) / (((data.groupby("horse")["horse"]).transform("cumcount")+1).apply(lambda x: 4 if x >= 4 else x))
data["h_cplace"] = (data.groupby("horse")["in_place"].transform("sum"))/(data.groupby("horse")["horse"].transform("count"))
data["h_rplace"] = (data.groupby("horse", group_keys=False)["in_place"].apply(lambda x: x.rolling(4, min_periods=1).sum())) / (((data.groupby("horse")["horse"]).transform("cumcount")+1).apply(lambda x: 4 if x >= 4 else x))
data["j_cwin"] = (data.groupby("jockey")["win"].transform("sum"))/(data.groupby("jockey")["jockey"].transform("count"))
data["j_rwin"] = (data.groupby("jockey", group_keys=False)["win"].apply(lambda x: x.rolling(4, min_periods=1).sum())) / (((data.groupby("jockey")["jockey"]).transform("cumcount")+1).apply(lambda x: 4 if x >= 4 else x))
data["j_cplace"] = (data.groupby("jockey")["in_place"].transform("sum"))/(data.groupby("jockey")["jockey"].transform("count"))
data["j_rplace"] = (data.groupby("jockey", group_keys=False)["in_place"].apply(lambda x: x.rolling(4, min_periods=1).sum())) / (((data.groupby("jockey")["jockey"]).transform("cumcount")+1).apply(lambda x: 4 if x >= 4 else x))

In [ ]:
data.to_csv("test.csv")